# Make Tests

Notebook for making different tests.

In [1]:
import json
import re

In [68]:
all_examples = {}
with open("../sample.en", "r") as f:
  lines = []
  for l in f:
    if (not len(l.strip())) or l.startswith("#"):
      continue
    lines.append(l)
  data = "".join(lines).strip().split(":::")[1:]
  
  for x in data:
    task = {
      "questions": [],
      "examples": []
    }
    name = x.split("\n")[0].strip().lower().replace(" ", "_")
    for l in x.split("\n")[1:]:
      if not l:
        continue
      if l[0] == "%":
        task["questions"].append(l[1:].strip())
      elif l[0] == "-":
        example = l[1:].strip()
        output = example.split("<>")
        if name != "words":
          input = output[0]
          try:
            options = [x.strip() for x in output[1].split(",")]
          except:
            print(name, output)
          task["examples"].append({
            "input": input.strip(),
            "options": options
          })
        
        # this is for words so only need the words
        else:
          task["examples"].append(example.strip())
    all_examples[name] = task
    

In [71]:
all_examples.keys()

dict_keys(['words', 'arxiv_classification_multiple', 'country_extraction_single'])

In [85]:
def nth_letter(data, n, na_tag = "NA"):
  # what is the n'th letter of the word
  nth_letters = []
  for x in data["examples"]:
    if len(x) > n:
      nth_letters.append(x[n])
    else:
      nth_letters.append(na_tag)
      
  # create sample strings
  strings = []
  for nth, word in zip(nth_letters, data["examples"]):
    strings.append(f"in -> {word}; out -> {nth}")
  
  return strings

In [87]:
zeroth_letters = nth_letter(all_examples["words"], 0)
zeroth_letters[:10]

['in -> information; out -> i',
 'in -> contact; out -> c',
 'in -> business; out -> b',
 'in -> services; out -> s',
 'in -> service; out -> s',
 'in -> products; out -> p',
 'in -> product; out -> p',
 'in -> available; out -> a',
 'in -> copyright; out -> c',
 'in -> support; out -> s']

In [97]:
nth_words_dataset = []
for n in range(10):
  samples = nth_letter(all_examples["words"], n)
  nth_words_dataset.append(samples)
  
nth_questions = []  
for n in range(10):
  qs = [
    f"What is the letter at {n}th location?",
    f"Output letter at {n}th location",
    f"{n}th letter?"
  ]
  nth_questions.append(qs)

In [100]:
with open("nth_letter.json", "w") as f:
  f.write(json.dumps({
    "questions": nth_questions,
    "samples": nth_words_dataset
  }))